<a href="https://colab.research.google.com/github/AtSourav/AE-w.-Bottleneck-Residual-Blocks/blob/main/AE_cifar10_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

For this notebook we shall use two custom python modules where we have defined some helper functions for plotting images, and more importantly some custom layers to implement residual blocks and in particular bottleneck residual blocks.

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow import math
import tensorflow_datasets as tfds
from keras import losses
from keras import layers
from keras import utils
from keras import metrics
from keras import backend as K
from keras import initializers

import os
import random
import matplotlib.pyplot as plt

# **Cloning the github repo with custom modules**

In [2]:
! git clone https://github.com/AtSourav/AE-w.-Bottleneck-Residual-Blocks

Cloning into 'AE-w.-Bottleneck-Residual-Blocks'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 82 (delta 34), reused 49 (delta 14), pack-reused 0
Receiving objects: 100% (82/82), 34.59 KiB | 3.84 MiB/s, done.
Resolving deltas: 100% (34/34), done.


In [3]:
%cd "/content/AE-w.-Bottleneck-Residual-Blocks"

/content/AE-w.-Bottleneck-Residual-Blocks


In [4]:
import plotting_tools as plts
import Bottleneck_residual_blocks as resblock

# **Set-up, and dataset (cifar10) loading**

In [5]:
img_ht = 32
img_wd = 32

input_size = (img_ht,img_wd,3)
latent_dim = 256
batch_size = 128

initializer = initializers.HeNormal(seed=100)

In [6]:
ds = tfds.load('cifar10', split='train')
ds2= tfds.load('cifar10', split='test')

ds_to_np = list(ds.as_numpy_iterator())
ds2_to_np = list(ds2.as_numpy_iterator())

img_train = np.array([x['image'] for x in ds_to_np])
img_valid = np.array([x['image'] for x in ds2_to_np])

img_train = img_train/255
img_valid = img_valid/255

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar10/3.0.2.incompleteML70S1/cifar10-train.tfrecord*...:   0%|          …

Generating test examples...:   0%|          | 0/10000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar10/3.0.2.incompleteML70S1/cifar10-test.tfrecord*...:   0%|          |…

Dataset cifar10 downloaded and prepared to /root/tensorflow_datasets/cifar10/3.0.2. Subsequent calls will reuse this data.


# **Building the model**

In this notebook, we shall stick to a compression ratio of 1, so we're not using bottleneck blocks.

In [48]:
# we're using a combination of max and min pooling, min pooling in the skip connections in the residual blocks, max pooling otherwise

# intuitively, max pooling should be ideal for lighter images on a dark background, min pooling for the opposite scenario

encoder_input = keras.Input(shape=input_size)

x = resblock.conv2d_block(64, 3, initializer, use_bn='False')(encoder_input)

x = resblock.conv2d_block(64, 2, initializer, use_bn='False')(x)

x = layers.MaxPooling2D(pool_size=(2, 2), strides=None, padding="valid")(x)

# -------------------------------------------------------------------------------

x = resblock.conv2d_block(128, 2, initializer, use_bn='False')(x)

x = resblock.bottleneck_residual_conv2D_block(128, 1, 2, initializer, 'min', use_bn='False')(x)

x = resblock.bottleneck_residual_conv2D_block(128, 1, 2, initializer, 'min', use_bn='False')(x)

x = resblock.bottleneck_residual_conv2D_block(128, 1, 2, initializer, 'min', use_bn='False')(x)

# -------------------------------------------------------------------------------

x = resblock.conv2d_block(256, 2, initializer, use_bn='False')(x)

x = resblock.bottleneck_residual_conv2D_block(256, 1, 2, initializer, 'min', use_bn='False')(x)

x = resblock.bottleneck_residual_conv2D_block(256, 1, 2, initializer, 'min', use_bn='False')(x)

x = resblock.bottleneck_residual_conv2D_block(256, 1, 2, initializer, 'min', use_bn='False')(x)

# -------------------------------------------------------------------------------

x = resblock.conv2d_block(512, 2, initializer, use_bn='False')(x)

x = resblock.bottleneck_residual_conv2D_block(512, 1, 2, initializer, 'min', use_bn='False')(x)

x = resblock.bottleneck_residual_conv2D_block(512, 1, 2, initializer, 'min', use_bn='False')(x)

#x = resblock.bottleneck_residual_conv2D_block(512, 1, 2, initializer, 'min', use_bn='False')(x)

# x = resblock.bottleneck_residual_conv2D_block(512, 1, 2, initializer, 'min', use_bn='False')(x)

# -------------------------------------------------------------------------------

x = layers.Flatten()(x)

x = layers.Dense(3*latent_dim)(x)
x = layers.ReLU()(x)

x = layers.Dense(2*latent_dim)(x)
x = layers.ReLU()(x)

z = layers.Dense(latent_dim, name="z")(x)

encoder = keras.Model(encoder_input, z, name='encoder')
encoder.summary()

Model: "encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d_block (conv2d_block  (None, 30, 30, 64)        1792      
 )                                                               
                                                                 
 conv2d_block_1 (conv2d_blo  (None, 29, 29, 64)        16448     
 ck)                                                             
                                                                 
 max_pooling2d (MaxPooling2  (None, 14, 14, 64)        0         
 D)                                                              
                                                                 
 conv2d_block_2 (conv2d_blo  (None, 13, 13, 128)       32896     
 ck)                                                       

In [49]:
latent_input = keras.Input(shape=(latent_dim,))

x = layers.Dense(2*latent_dim)(latent_input)
x = layers.ReLU()(x)

x = layers.Dense(3*latent_dim)(x)
x = layers.ReLU()(x)

x = layers.Dense(4*latent_dim)(x)
x = layers.ReLU()(x)

x = layers.Dense(2*2*1024)(x)
x = layers.ReLU()(x)

# -------------------------------------------------------------------------------

x = layers.Reshape((2,2,1024))(x)

x = resblock.conv2dtrans_block(1024, 1, initializer, use_bn='False')(x)

# -------------------------------------------------------------------------------

x = resblock.conv2dtrans_block(512, 1, initializer, use_bn='False')(x)

x = resblock.bottleneck_residual_conv2Dtrans_block(512, 1, 3, initializer, use_bn='False')(x)

# -------------------------------------------------------------------------------

x = resblock.conv2dtrans_block(256, 3, initializer, use_bn='False')(x)

x = resblock.bottleneck_residual_conv2Dtrans_block(256, 1, 2, initializer, use_bn='False')(x)

#x = resblock.bottleneck_residual_conv2Dtrans_block(256, 1, 2, initializer, use_bn='False')(x)

# -------------------------------------------------------------------------------

x = layers.UpSampling2D(size=(2, 2), data_format=None, interpolation='bilinear')(x)

# -------------------------------------------------------------------------------

x = resblock.conv2dtrans_block(128, 3, initializer, use_bn='False')(x)

x = resblock.bottleneck_residual_conv2Dtrans_block(128, 1, 3, initializer, use_bn='False')(x)

#x = resblock.bottleneck_residual_conv2Dtrans_block(128, 1, 2, initializer, use_bn='False')(x)

#x = resblock.bottleneck_residual_conv2Dtrans_block(128, 1, 2, initializer, use_bn='False')(x)

# -------------------------------------------------------------------------------

x = layers.UpSampling2D(size=(2, 2), data_format=None, interpolation='bilinear')(x)

# -------------------------------------------------------------------------------

#x = resblock.bottleneck_residual_conv2D_block(128, 1, 2, initializer, 'min', use_bn='False')(x)

x = resblock.bottleneck_residual_conv2D_block(128, 1, 2, initializer, 'min', use_bn='False')(x)

x = resblock.bottleneck_residual_conv2D_block(128, 1, 2, initializer, 'min', use_bn='False')(x)

# -------------------------------------------------------------------------------

x = resblock.conv2d_block(64, 2, initializer, use_bn='False')(x)

x = resblock.conv2d_block(64, 2, initializer, use_bn='False')(x)

# -------------------------------------------------------------------------------

decoder_output = layers.Conv2D(3, 1, activation='sigmoid', padding='valid', kernel_initializer=initializer)(x)

decoder = keras.Model(latent_input, decoder_output, name="decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 256)]             0         
                                                                 
 dense_2 (Dense)             (None, 512)               131584    
                                                                 
 re_lu_31 (ReLU)             (None, 512)               0         
                                                                 
 dense_3 (Dense)             (None, 768)               393984    
                                                                 
 re_lu_32 (ReLU)             (None, 768)               0         
                                                                 
 dense_4 (Dense)             (None, 1024)              787456    
                                                                 
 re_lu_33 (ReLU)             (None, 1024)              0   

In [45]:
decoder_out = decoder(encoder(encoder_input))
AE = keras.Model(encoder_input, decoder_out, name='AE')

AE.summary()

Model: "AE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 encoder (Functional)        (None, 256)               9332544   
                                                                 
 decoder (Functional)        (None, 32, 32, 3)         12267331  
                                                                 
Total params: 21599875 (82.40 MB)
Trainable params: 21599875 (82.40 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# **Training**

In [50]:
K.clear_session()

In [ ]:
optim = keras.optimizers.Adam(learning_rate = 0.0001)
AE.compile(optimizer=optim,loss='mse')
history = AE.fit(img_train, img_train, batch_size=batch_size, validation_data=(img_valid, img_valid), epochs=15, steps_per_epoch=None)

Epoch 1/15
391/391 [==============================] - 48s 82ms/step - loss: 0.0197 - val_loss: 0.0164
Epoch 2/15
391/391 [==============================] - 31s 80ms/step - loss: 0.0156 - val_loss: 0.0158
Epoch 3/15
391/391 [==============================] - 32s 81ms/step - loss: 0.0150 - val_loss: 0.0149
Epoch 4/15
391/391 [==============================] - 31s 80ms/step - loss: 0.0144 - val_loss: 0.0142
Epoch 5/15
391/391 [==============================] - 31s 80ms/step - loss: 0.0138 - val_loss: 0.0140
Epoch 6/15
391/391 [==============================] - 31s 80ms/step - loss: 0.0135 - val_loss: 0.0139
Epoch 7/15
391/391 [==============================] - 31s 80ms/step - loss: 0.0131 - val_loss: 0.0131
Epoch 8/15
391/391 [==============================] - 31s 80ms/step - loss: 0.0129 - val_loss: 0.0130
Epoch 9/15
391/391 [==============================] - 31s 80ms/step - loss: 0.0126 - val_loss: 0.0131
Epoch 10/15
391/391 [==============================] - 31s 80ms/step - loss: 0.012

In [37]:
optim = keras.optimizers.Adam(learning_rate = 0.00001)
AE.compile(optimizer=optim,loss='mse')
history2 = AE.fit(img_train, img_train, batch_size=batch_size, validation_data=(img_valid, img_valid), epochs=15, steps_per_epoch=None)

Epoch 1/15
391/391 [==============================] - 50s 85ms/step - loss: 0.0096 - val_loss: 0.0101
Epoch 2/15
391/391 [==============================] - 32s 81ms/step - loss: 0.0095 - val_loss: 0.0101
Epoch 3/15
391/391 [==============================] - 32s 81ms/step - loss: 0.0095 - val_loss: 0.0101
Epoch 4/15
391/391 [==============================] - 32s 81ms/step - loss: 0.0095 - val_loss: 0.0100
Epoch 5/15
391/391 [==============================] - 32s 81ms/step - loss: 0.0095 - val_loss: 0.0100
Epoch 6/15
351/391 [=========================>....] - ETA: 3s - loss: 0.0094

KeyboardInterrupt: ignored